In [1]:
!pip install transformers datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
def read_conll_data(file_path):
    sentences = []  # Initialize a list to store sentences
    current_tokens = []  # Temporary list for the current sentence's tokens
    current_labels = []  # Temporary list for the current sentence's labels

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()  # Remove leading/trailing whitespace

            if line == "":  # Check for the end of a sentence
                if current_tokens:  # If there are tokens collected
                    sentences.append((current_tokens, current_labels))  # Add the current tokens and labels to the sentences list
                    current_tokens = []  # Reset for the next sentence
                    current_labels = []  # Reset for the next sentence
            else:
                token_label_pair = line.split()  # Split the line into token and label
                if len(token_label_pair) == 2:  # Ensure there are exactly two elements
                    token, label = token_label_pair  # Unpack the token and label
                    current_tokens.append(token)  # Add the token to the current list
                    current_labels.append(label)  # Add the label to the current list

        # After processing all lines, add the last sentence if it was not followed by a blank line
        if current_tokens:
            sentences.append((current_tokens, current_labels))

    return sentences  # Return the collected sentences and labels

# Example usage:
file_path = "/content/drive/My Drive/labeled_telegram_product_price_location.coll"  # Path to the CoNLL file
conll_data = read_conll_data(file_path)  # Read the data from the file

# Print the first sentence along with its tokens and labels
print(conll_data[0])  # Example output: (['token1', 'token2'], ['label1', 'label2'])


(['️', 'ዘመናዊ', 'በኤሌክትሮኒክስ', 'የሚሰራ', 'ሚዛን', '️እስከ', '50', 'ኪሎ', 'ይመዝናል', '️', 'በባትሪ', 'ድንጋይ', 'የሚሰራ', '️', 'ምንም', 'የማያዛንፍ', 'ሃቀኛ', 'ሚዛን', '️', 'በቀላሉ', 'በኪስ', 'የሚያዝ', '️', 'ውጭ', 'ሀገር', 'ዕቃ', 'ለሚልኩ', 'እና', 'ለተንቀሳቃሽ', 'ነጋዴዎች', 'ይፈለጋል', '900', 'ብር', 'ሱቃችን', 'መጥተው', 'መኪና', 'ማቆሚያ', 'ከተቸገሩ', 'መኪናዎ', 'ላይ', 'ሆነው', 'ይደውሉ', 'እናቀብልዎታለን', 'ከ100', 'እስከ', '200', 'ብር', 'ብቻ', 'ከፍለው', 'ካሉበት', 'እንልካለን', 'ክፍለሃገር', 'ላላችሁ', 'ደንበኞቻችን', 'የፈለጉትን', 'ዕቃ', 'በመልዕክት', 'እንልክልዎታለን', 'ለማዘዝ', '@1', 'ለማዘዝ', '@2', 'ለወዳጅዎ', 'በማድረግ', 'ይተባበሩን', '️', '0944-22-23-24', '️', '0904-94-48-48', 'አድራሻችን', 'መገናኛ', 'ዘፍመሽ', 'ግራንድ', 'ሞል', '3ኛ', 'ፎቅ', 'ከሊፍት', 'ሲወርዱ', 'ወደ', 'ቀኝ', 'ታጥፈው', 'ቀጥታ', '376\u200c', 'በኪስዎ', 'ጥሬ', 'ገንዘብ', 'ካልያዙ', 'በሞባይል', 'ማስተላለፍ', 'ይችላሉ።', 'ይሄንን', './', 'ተጭነው', 'ያድርጉ፣', 'ቤተሰብ', 'ይሁኑ'], ['B-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT', 'I-PRODUCT

In [5]:
import pandas as pd

In [6]:
# Create DataFrame
tokens, labels = zip(*conll_data)
df = pd.DataFrame({'tokens': tokens, 'labels': labels})

df

,tokens,labels
0,"[️, ዘመናዊ, በኤሌክትሮኒክስ, የሚሰራ, ሚዛን, ️እስከ, 50, ኪሎ, ...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
1,"[️, ቦርጭ, የሚሰበስብ, ️የቦርጭ, ስብ, የሚያቃጥል, ️, ማራኪ, የሰ...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
2,"[ይሄንን, ./, ተጭነው, ያድርጉ፣, ቤተሰብ, ይሁኑ]","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
3,"[️, ባለ, ዘርፈ, ብዙ, ጥቅም, ️, ለጫማ፣, ለባርኔጣ፡ኮፍያ, መሰል,...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
4,"[ይሄንን, ./, ተጭነው, ያድርጉ፣, ቤተሰብ]","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
...,...,...
3620,[2500],[B-PRODUCT]
3621,"[ዋጋ, 2500, 0983063957]","[B-PRODUCT, I-PRODUCT, I-PRODUCT]"
3622,"[የሙያ, ባለቤት, መሆን, መሠልጠን, ነው።, ቀለም, ቀቢ, ሳያስፈልግዎ,...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
3623,"[ቤትና, ግቢዎን, እንዲሁም, የብረት, እና, የእንጨት, ቁሳቁስዎን, ቀለ...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."


In [7]:

# Example of label encoding
label_encoding = {'O': 0, 'B-PRODUCT' or 'B-Product': 1, 'I-PRODUCT': 2, 'I-PHONE': 3, 'I-LOC': 4, 'I-PRICE': 5}  # Add all your label mappings here

# Apply label encoding to the 'labels' column
df['labels'] = df['labels'].apply(lambda x: [label_encoding[label] for label in x])


In [8]:

import ast  # For safely evaluating list strings if needed
from datasets import load_dataset,Dataset

# Ensure that 'tokens' and 'labels' columns are lists
df['tokens'] = df['tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['labels'] = df['labels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df, preserve_index=False)


In [9]:
# Split the dataset into training and remaining sets (80% train, 20% remaining)
train_test_split = dataset.train_test_split(test_size=0.2)  # 80% train, 20% remaining
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Further split the remaining dataset into validation and test sets (50% of remaining for each)
#val_test_split = remaining_dataset.train_test_split(test_size=0.5)  # 50% for validation, 50% for testing
#eval_dataset = val_test_split['train']  # Validation set
#test_dataset = val_test_split['test']    # Test set

# Output the sizes of the datasets
print(f"Training dataset size: {len(train_dataset)}")
#print(f"Validation dataset size: {len(eval_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Training dataset size: 2900
Test dataset size: 725


In [10]:
unique_labels = set()
for example in train_dataset:
    unique_labels.update(example['labels'])
print("Unique labels in the dataset:", unique_labels)


Unique labels in the dataset: {1, 2}


In [11]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

label_all_tokens = False
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens like [CLS], [SEP]
            elif word_idx != previous_word_idx:
                if word_idx < len(label):  # Ensure valid index for the label
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)  # Handle index out of bounds
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
#tokenized_eval_dataset = eval_dataset.map(tokenize_and_align_labels, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/2900 [00:00<?, ? examples/s]

Map:   0%|          | 0/725 [00:00<?, ? examples/s]

In [17]:
from transformers import AutoModelForTokenClassification

# Load a pre-trained token classification model (XLM-RoBERTa)
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

def custom_collate_fn(batch):
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    attention_mask = [torch.tensor(item['attention_mask']) for item in batch]
    labels = [torch.tensor(item['labels']) for item in batch]

    padded_input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)

    max_length = padded_input_ids.size(1)
    padded_labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
    padded_labels = torch.nn.functional.pad(padded_labels, (0, max_length - padded_labels.size(1)), value=-100)

    # Debugging shapes and values
    # print(f'Batch input_ids shape: {padded_input_ids.shape}')
    # print(f'Batch attention_mask shape: {padded_attention_mask.shape}')
    # print(f'Batch labels shape: {padded_labels.shape}')
    # print(f'Batch labels values: {padded_labels}')  # Inspect the actual label values

    return {
        'input_ids': padded_input_ids,
        'attention_mask': padded_attention_mask,
        'labels': padded_labels,
    }

# Create DataLoader with the custom collate function
train_loader = DataLoader(tokenized_train_dataset, batch_size=16, shuffle=True, collate_fn=custom_collate_fn)
eval_loader = DataLoader(tokenized_test_dataset, batch_size=16, collate_fn=custom_collate_fn)
# Define a custom Trainer that accepts DataLoaders

# Setup training arguments
# Setup training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

class CustomTrainer(Trainer):
    def get_train_dataloader(self):
        return train_loader

    def get_eval_dataloader(self, eval_dataset=None):
        return eval_loader


# Initialize the CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("/content/drive/My Drive/DistillBert_fine_tuned_model")
tokenizer.save_pretrained("/content/drive/My Drive/DistillBert_fine_tuned_tokenizer")

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.008600,0.007702
2,0.006800,0.007213
3,0.006400,0.006871


('/content/drive/My Drive/DistillBert_fine_tuned_tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/DistillBert_fine_tuned_tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/DistillBert_fine_tuned_tokenizer/vocab.txt',
 '/content/drive/My Drive/DistillBert_fine_tuned_tokenizer/added_tokens.json',
 '/content/drive/My Drive/DistillBert_fine_tuned_tokenizer/tokenizer.json')

Evaluate the model

In [18]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 0.0068714814260602, 'eval_runtime': 7.5619, 'eval_samples_per_second': 95.876, 'eval_steps_per_second': 6.083, 'epoch': 3.0}


In [19]:
def get_predictions(trainer, eval_loader):
    model.eval()  # Set model to evaluation mode
    predictions, true_labels = [], []

    for batch in eval_loader:
        with torch.no_grad():  # Disable gradient calculation
            outputs = model(**{k: v.to(trainer.model.device) for k, v in batch.items()})
            logits = outputs.logits

        predicted = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = batch['labels'].cpu().numpy()

        # Append to lists
        predictions.append(predicted)
        true_labels.append(labels)

    return predictions, true_labels


In [21]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
# Get predictions from the evaluation set
predictions, true_labels = get_predictions(trainer, eval_loader)

# Initialize lists to hold filtered predictions and true labels
true_labels_flat = []
predictions_flat = []

# Flatten the labels and predictions for each sequence in the batch
for batch_labels, batch_preds in zip(true_labels, predictions):
    # Flatten the true labels and predictions
    batch_labels_flat = batch_labels.flatten()  # Flatten to 1D
    batch_preds_flat = batch_preds.flatten()    # Flatten to 1D

    # Only keep predictions corresponding to valid labels
    for label, pred in zip(batch_labels_flat, batch_preds_flat):
        if label != -100:  # Only collect predictions for valid labels
            true_labels_flat.append(label)
            predictions_flat.append(pred)

# Convert to numpy arrays for sklearn metrics
true_labels_flat = np.array(true_labels_flat)
predictions_flat = np.array(predictions_flat)

# Debugging: Check shapes and types
print(f"Shape of true labels: {true_labels_flat.shape}")
print(f"Shape of predicted labels: {predictions_flat.shape}")
print(f"True labels (first 10): {true_labels_flat[:10]}")
print(f"Predictions (first 10): {predictions_flat[:10]}")

# Calculate metrics only if lengths match
if len(true_labels_flat) == len(predictions_flat):
    accuracy = accuracy_score(true_labels_flat, predictions_flat)
    f1 = f1_score(true_labels_flat, predictions_flat, average='weighted')
    precision = precision_score(true_labels_flat, predictions_flat, average='weighted')
    recall = recall_score(true_labels_flat, predictions_flat, average='weighted')

    # Print metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
else:
    print("Length mismatch: Cannot calculate metrics.")

Shape of true labels: (33659,)
Shape of predicted labels: (33659,)
True labels (first 10): [1 2 2 2 2 2 2 2 2 2]
Predictions (first 10): [1 2 2 2 2 2 2 2 2 2]
Accuracy: 0.9966
F1 Score: 0.9966
Precision: 0.9966
Recall: 0.9966


In [23]:
pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 10.3 MB/s eta 0:00:00


Model Interpretability

In [28]:
import shap
import torch
import numpy as np

# Function to prepare input for SHAP
def predict_fn(input_ids):
    # Prepare attention mask
    attention_mask = (input_ids != tokenizer.pad_token_id).astype(int)

    # Ensure model is in evaluation mode
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=torch.tensor(input_ids), attention_mask=torch.tensor(attention_mask))

    # Return logits
    logits = outputs.logits.numpy()  # Shape: (batch_size, sequence_length, num_labels)
    print("Logits shape from predict_fn:", logits.shape)  # Debugging output
    return logits.reshape(logits.shape[0], -1)  # Reshape to (batch_size, sequence_length * num_labels)

# Sample input text
input_text = "ምን መግዛት ይፈልጋሉ"
tokens = tokenizer(input_text, return_tensors='pt')

# Prepare background data with padding
background_sentences = [
    "This is a sample sentence.",
    "Another example sentence.",
    "Yet another sentence."
]

# Tokenize and pad background sentences
background_data = tokenizer(background_sentences, padding=True, truncation=True, return_tensors='pt')

# Convert to NumPy array
background_input_ids = background_data['input_ids'].numpy()

# Create a SHAP Kernel Explainer
explainer = shap.KernelExplainer(predict_fn, background_input_ids)

# Calculate SHAP values for a single input text
input_ids_np = tokens['input_ids'].numpy()

# Ensure input has correct dimensions
if input_ids_np.ndim == 1:
    input_ids_np = input_ids_np.reshape(1, -1)  # Reshape to (1, sequence_length)

# Print shapes for debugging
print("Input IDs shape:", input_ids_np.shape)

# Calculate SHAP values
shap_values = explainer.shap_values(input_ids_np, nsamples=100)  # Adjust sample size as needed


Logits shape from predict_fn: (3, 8, 6)
Input IDs shape: (1, 5)


  0%|          | 0/1 [00:00<?, ?it/s]

IndexError: index 5 is out of bounds for axis 1 with size 5

In [29]:
# Get the expected value
expected_value = explainer.expected_value  # This may be a list if there are multiple outputs
print("Expected value shape:", np.shape(expected_value))  # Debugging output


Expected value shape: (48,)


In [32]:
# Sample input text
input_text = "ምን መግዛት ይፈልጋሉ"

# Tokenize the input text
tokens = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)

# Print token IDs and decoded tokens
print("Token IDs:", tokens['input_ids'])
print("Decoded Tokens:", tokenizer.convert_ids_to_tokens(tokens['input_ids'][0].numpy()))

Token IDs: tensor([[101, 100, 100, 100, 102]])
Decoded Tokens: ['[CLS]', '[UNK]', '[UNK]', '[UNK]', '[SEP]']
